# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [2]:
countries=[]
infant_mortality=[]
document = ET.parse( './data/mondial_database.xml' ) #parse the doc
for element in document.iterfind('country'):
    countries.append(element.find('name').text)
    try:
        infant_mortality.append(element.find("infant_mortality").text)
    except AttributeError:
        infant_mortality.append("Nan")
#create lists of countreis and infant mortality

In [3]:
tot=zip(countries, infant_mortality)
tot=pd.DataFrame(tot) #make a dataframe
tot.sort_values(0).set_index(keys=0).reset_index().head() #organize the dataframe alphabetically

,0,1
0,Afghanistan,117.23
1,Albania,13.19
2,Algeria,21.76
3,American Samoa,8.92
4,Andorra,3.69


Top Ten Countries, based on infant mortality rate, descending

In [77]:
tot[1]=tot[1].convert_objects(convert_numeric=True)
tot.sort_values(1).head(10)

C:\Users\Nathan\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


,0,1
38,Monaco,1.81
98,Japan,2.13
117,Bermuda,2.48
36,Norway,2.48
106,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


In [17]:
city_list=[]
for element in document.iterfind('country'):
    for subelement in element.getiterator('city'):
        city_list.append(subelement.find('name').text) #create a list of cities

In [15]:
pops=[]
for element in document.iterfind('country'):
    for subelement in element.getiterator('city'):
        i=subelement.findall('population')
        y=(len(i))-1
        try:
            pops.append(i[y].text)
        except IndexError:
            pops.append(np.nan)

In [18]:
citypop=pd.DataFrame(zip(city_list, pops))
citypop.sort_values(0).set_index(keys=0).reset_index().head() #sort alphabetically and create a dataframe

,0,1
0,'s-Hertogenbosch,143822
1,A Coruña,245053
2,Aachen,236420
3,Aalborg,104885
4,Aarau,NaN


# Ten Largest Cities in the Database

In [19]:
citypop[1]=citypop[1].convert_objects(convert_numeric=True)
citypop.sort_values(1, ascending=False).head(10) #find the ten largest cities in the database

C:\Users\Nathan\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


,0,1
1341,Shanghai,22315474.0
771,Istanbul,13710512.0
1527,Mumbai,12442373.0
479,Moskva,11979529.0
1340,Beijing,11716620.0
2810,São Paulo,11152344.0
1342,Tianjin,11090314.0
1064,Guangzhou,11071424.0
1582,Delhi,11034555.0
1067,Shenzhen,10358381.0


# # Ten Largest Ethnic Groups, By Population

In [9]:
ethnicgroup=[]
egpercentage=[]
country=[]
totalpop=[]
for element in document.iterfind('country'):
    for subelement in element.getiterator('ethnicgroup'):
        ethnicgroup.append(subelement.text)
        egpercentage.append(subelement.get("percentage"))
        country.append(element.find("name").text)
        i=element.findall("population")
        y=len(i)-1
        totalpop.append(i[y].text)


In [14]:
top=["Ethnic Group", "Percentage", "Country", "Total_Population"]
ethnic=pd.DataFrame(zip(ethnicgroup, egpercentage, country, totalpop), columns=top)
ethnic.head()

,Ethnic Group,Percentage,Country,Total_Population
0,Albanian,95,Albania,2800138
1,Greek,3,Albania,2800138
2,Greek,93,Greece,10816286
3,Macedonian,64.2,Macedonia,2059794
4,Albanian,25.2,Macedonia,2059794


In [17]:
ethnic.Percentage=pd.to_numeric(ethnic.Percentage)
ethnic.Total_Population=pd.to_numeric(ethnic.Total_Population)
ethnic.Percentage=ethnic.Percentage/100

In [18]:
ethnic["totalethnicgrouppop"]=ethnic.Percentage*ethnic.Total_Population
ethnic.sort_values(by="totalethnicgrouppop", ascending=False).head(10)

,Ethnic Group,Percentage,Country,Total_Population,totalethnicgrouppop
176,Han Chinese,0.009150,China,1360720000,1.245059e+07
221,Indo-Aryan,0.007200,India,1210854977,8.718156e+06
220,Dravidian,0.002500,India,1210854977,3.027137e+06
345,European,0.007996,United States,318857056,2.549581e+06
520,African,0.009900,Nigeria,164294516,1.626516e+06
212,Bengali,0.009800,Bangladesh,149772364,1.467769e+06
299,Japanese,0.009940,Japan,127298000,1.265342e+06
93,Russian,0.007980,Russia,143666931,1.146462e+06
278,Javanese,0.004500,Indonesia,252124458,1.134560e+06
461,European,0.005370,Brazil,202768562,1.088867e+06


# Longest River, Largest Lake, Highest Airport

In [154]:
river_list=[]
for element in document.iterfind("river"):
    country_codes=element.get("country")  #find country codes
    river_names=element.find("name").text #find river names
    try:
        river_length=element.find("length").text #worry about potential errors
    except AttributeError:
        river_length=np.nan
    for river_code in element.attrib["country"].split():
        river_country_code= keydic[river_code]  #rivers run through multiple countries, thus need to split them and treat each part independently for purposes of this exercise 
    river_list.append([river_country_code, river_names, river_length])
#print(river_list)

In [106]:
car_codes=[]
country_names=[]
for element in document.iterfind('country'):
    car_codes.append(element.get("car_code"))
    country_names.append(element.find("name").text)
keydic=dict(zip(car_codes, country_names))  #build a dic of countries/country codes

In [161]:

top=["country", "name", "length"]
rivers=pd.DataFrame(river_list, columns=top)
rivers.head() #make a rivers data frame

,country,name,length
0,Iceland,Thjorsa,230
1,Iceland,Joekulsa a Fjoellum,206
2,Norway,Glomma,604
3,Norway,Lagen,322
4,Sweden,Goetaaelv,93


# Longest River

In [158]:
rivers.length=pd.to_numeric(rivers.length)
rivers.sort_values(by="length", ascending=False).head(1) #find the longest river

,code,name,length
174,Peru,Amazonas,6448.0


In [160]:
lake_list=[]
for element in document.iterfind("lake"):
    lake_names=element.find("name").text
    try:
        lake_area=element.find("area").text #make a list of lakes
    except AttributeError:
        lake_area=np.nan
    for lake_code in element.attrib["country"].split():
        lake_country_code= keydic[lake_code]
    lake_list.append([lake_country_code, lake_names, lake_area])
#print(lake_list) 

In [162]:
top=["country", "lake_name", "area"]
lakes=pd.DataFrame(lake_list, columns=top)
lakes.head() #build a lake database

,country,lake_name,area
0,Finland,Inari,1040
1,Finland,Oulujaervi,928
2,Finland,Kallavesi,472
3,Finland,Saimaa,4370
4,Finland,Paeijaenne,1118


# Largest Lake

In [164]:
lakes.area=pd.to_numeric(lakes.area)
lakes.sort_values(by="area", ascending=False).head(1)

,country,lake_name,area
54,Turkmenistan,Caspian Sea,386400.0


In [166]:
airport_list=[]
for element in document.iterfind("airport"):
    airport_names=element.find("name").text
    airportelev=element.find("elevation").text  #build a list of airports
    for code in element.attrib["country"].split():
        country_code= keydic[code]
    airport_list.append([country_code, airport_names, airportelev])
#print(airport_list)

In [167]:
top=["country", "Airport Name", "Elevation"]
airport=pd.DataFrame(airport_list, columns=top)
airport.head()  #make an airport dataframe

,country,Airport Name,Elevation
0,Afghanistan,Herat,977
1,Afghanistan,Kabul Intl,1792
2,Albania,Tirana Rinas,38
3,Algeria,Cheikh Larbi Tebessi,811
4,Algeria,Batna Airport,822


# Airport at the Highest Elevation

In [169]:
airport.Elevation=pd.to_numeric(airport.Elevation)
airport.sort_values(by="Elevation", ascending=False).head(1)

,country,Airport Name,Elevation
80,Bolivia,El Alto Intl,4063.0
